# Bayesian Batch Adaptation Using "Cutting the Feedback"

#### Author:  Austin Talbot



In this notebook we demonstrate usage of our new method for batch adaptation using Bayesian statistics combined with a technique called cutting the feedback to make the method robust to non-balanced data (our method won't fail if somebody unfortunately puts all controls in one batch and all cases in another batch). But before we explain how our method works, we first describe the setup/assumptions



## Setup and assumptions

### Background
We assume that our data come from $J$ batches and for each batch we have $I_j$ samples which we denote $\{x_{ij}\}_{i=1:I_j}$. Since this is designed around proteomics data $J$ will be relatively large (at least 20ish) while $I_j$ will be relatively small (around 15). These measurements have $p$ covariates and are normally distributed. The problem we face is that each batch has additive noise which we denote $\theta_i\sim N(0,\Sigma_\theta)$, which corrupt our observations. Let our signal in reality be $\delta_{ij}$, which we estimate. Then our observational model is 
\begin{equation}
\begin{split}
x_{ij}=\delta_{ij} + \theta_i + \epsilon_{ij}, \epsilon_{ij}\sim N(0,\Sigma_\epsilon).
\end{split}
\end{equation}

Methods such as COMBAT simply subtract off the mean of the batch as the way of adjusting for batch effects. However, there are two problems with this. (1) With small sample sizes this is throwing out a substantial portion of the information (in an extreme case with $I_j=1$ you lose the entire signal. And worse, (2) if the data are not balanced (i.e. different numbers of cases and controls) the sample mean is going to be correlated with the main signal we want to estimate (how does illness impact the covariates). To translate to statistics, $\delta_{ij}$ are not $iid$.

Normally, we would be up a creek without a paddle with this kind of question. However, we do have one saving grace, in each batch we also record the output of a control sample $y_i$, which is measured in all batches. This is equivalent to saying that $\delta_{i}=c$, which means that
\begin{equation}
y_i = c + \theta_i + \epsilon_i,\epsilon_{i}\sim N(0,\Sigma_\epsilon)
\end{equation}
We can pretend $c$ is 0 by subtracting off the control mean. Roughly true if enough samples are measured.

The innate tempation is to simply subtract off the value of this control to account for batch effects. This is nice because the resulting value is an unbiased estimator of $\delta{ij}$. Proof:

\begin{equation}
E[x_{ij}-y_i] = E[\delta_{ij} + \theta_i + \epsilon_{ij} - \theta_i - \epsilon_i]=\delta_{ij} +E[\epsilon_{ij}-\epsilon_i] = \delta_{ij}
\end{equation}

However, this estimator of $\delta_{ij}$ has a higher variance than $x_{ij}$, in fact it doubles the variance. In the extreme case that there are no batch effects, you've essentially doubled the measurement error.

### Our method
Instead, the optimal estimator for $\delta_{ij}$ is 
\begin{equation}
\tilde{x}_{ij} = x_{ij} - w y_i,
\end{equation}
where $w\in[0,1]$. I'm not going to derive the formula but in the univeriate case $w=\sigma^2_\theta/(\sigma^2_\theta+\sigma_\epsilon^2)$. If we knew $\Sigma_\epsilon,\Sigma_\theta$ this would be straightforward and solved. However, we must estimate them given the data. Once we put priors on our parameters we can be traditional Bayesians for this problem. Our model is

\begin{equation}
\begin{split}
p(\Sigma_\epsilon)&=Wishart(\nu+p,I)\\
p(\Sigma_\delta)&=Wishart(\nu+p,I)\\
p(\Sigma_\theta)&=InverseWishart(\nu+p,I)\\
p(\theta_i|\Sigma_\theta)&=N(0,\Sigma_\theta)\\
p(y_i|\theta_i,\Sigma_\epsilon)&=N(0,\Sigma_\theta)\\
p(x_{ij}|\theta_i,\Sigma_\epsilon,\delta_{ij})&=N(\theta_i+\delta_{ij},\Sigma_\epsilon)\\
\end{split}
\end{equation}


### Cutting the feedback

There is one major problem from the above formulation, we do not want $x_{ij}$ to assume iid. To solve this issue we will use a technique called cutting the feedback. Note that our posterior $p(\theta_i|-)$ will naturally depend on $x_{ij}$, as the samples are informative of the batch effects. However, in our sampling scheme we will cut this dependence and only condition $\theta_i$ on $y_i$. This is known as cutting the feedback. This substantially reduces our certainty on $\theta_i$, particularly if we were to have large batch sizes. And it makes samplers a real pain, which is why $\Sigma_\theta$ is the odd ball out. However, we will benefit enormously in an application where all the controls are put in a single batch.



## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from bystro.domain_adaptation.batch_bayesian import BatchAdaptationBayesian

/home/austin/anaconda3/envs/bystro/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Generative Mechanism for Data Simulation

1. **Batch Effects**:  
   A total of `n_batches` are generated from a multivariate normal distribution with mean zero and covariance matrix `sig_theta * I_p`, where `I_p` is the identity matrix of size `p`:
  $
   \text{Batch Effects} \sim \mathcal{N}(0, \sigma_{\theta}^2 I_p)
   $

2. **Controls**:  
   The controls are generated by adding Gaussian noise with variance `sig_eps^2` to the batch effects for each batch:
   $
   \text{Controls}_i = \text{Batch Effects}_i + \sigma_{\epsilon} \mathcal{N}(0, 1)
   $
   for each batch $(i = 1, \dots, n_{\text{batches}})$.

3. **True Values**:  
   For each batch, the true values are drawn from a normal distribution with variance `sig_delta^2`:
   $
   \text{True Values}_i \sim \sigma_{\delta} \mathcal{N}(0, 1)
   $
   for each sample within a batch \(i = 1, \dots, n_{\text{batches}}\).

4. **Observed Data**:  
   The observed data for each batch is generated by adding the batch effect and Gaussian noise with variance `sig_eps^2` to the true values:
   $
   X_i = \text{True Values}_i + \text{Batch Effects}_i + \sigma_{\epsilon} \mathcal{N}(0, 1)
   $
   for each batch $(i = 1, \dots, n_{\text{batches}})$.


In [2]:
def simulate_data(n_batches=30, n_samples=15, p=3, seed=1993,sig_theta=1.0,sig_eps=1.0,sig_delta=1.0):
    rng = np.random.default_rng(seed)
    batch_effects = rng.multivariate_normal(
        mean=np.zeros(p), cov=sig_theta * np.eye(p), size=n_batches
    )   

    controls = batch_effects + sig_eps*rng.normal(size=(n_batches, p)) 
    true_vals = [sig_delta*rng.normal(size=(n_samples, p)) for i in range(n_batches)]
    X_list = [ 
        true_vals[i] + batch_effects[i] + sig_eps*rng.normal(size=(n_samples, p)) 
        for i in range(n_batches)
    ]   
    return X_list, controls, true_vals, batch_effects


## Steps for our Bayesian Batch Adaptation Model

### 1. Creating a BatchAdaptationBayesian Object
In Bystro (or similarly in scikit-learn), the first step is to choose the BatchAdaptationBayesian method for imputing missing data. This model corresponds to PCA from scikit learn. Any training options go in the creation method.

### 2. Fit Model Using the `fit_transform()` Method
Once you have your PPCADropout object, the next step is to fit the model given data. You do this by calling the `fit_transform()`. This will estimate $\Sigma_\epsilon,\Sigma_\theta,w$ and return the adapted measurments. `fit_transform` takes 2 inputs, a list of matrices corresponding to the measurments from your experiment, and a matrix of control samples. 

### 3. Transforming new data
After the `fit_transform()` method has been applied, you can account for batch effects in new data using the `transform()` method. This takes a new list and matrix of controls. Unlike `fit_transform`, it uses the previously-computed 

### Putting It All Together
The process with SoftImpute is straightforward:

- **Create**: You start by creating a BatchAdaptationBayesian object that will learn your model
- **Fit_transform**: You then adjust for batch effects
- **transform**: Remove batch effects in new data



## Demonstrating usage

We now demonstrate usage on synthetic data and compare our performance to ignoring batch effects and subtracting off the control samples.

In [3]:
def run_experiment(seed,sig_theta=1.0,sig_eps=1.0):
    X_list, controls, true_vals, batch_effects = simulate_data(seed=seed,sig_theta=sig_theta,sig_eps=sig_eps)

    model = BatchAdaptationBayesian(n_burn=1000,n_samples=1000)
    data_altered = model.fit_transform(X_list, controls)

    data_altered_stack = np.vstack(data_altered)
    data_original_stack = np.vstack(X_list)
    true_stack = np.vstack(true_vals)

    data_subtracted = [X_list[i] - controls[i] for i in range(len(X_list))]
    data_subtracted_stack = np.vstack(data_subtracted)

    error_adapted = np.mean((data_altered_stack - true_stack) ** 2)
    error_original = np.mean((data_original_stack - true_stack) ** 2)
    error_subtracted = np.mean((data_subtracted_stack - true_stack) ** 2)
    print('Ours',error_adapted)
    print('Ignored',error_original)
    print('Naive',error_subtracted)
    print(model.w)

    return model,X_list, controls, true_vals, batch_effects,data_subtracted


## Comparable measurement noise and batch effects

In our first example, we show what happens when the measurement noise is comparable to the batch effects (both $\Sigma_\epsilon,\Sigma_\theta=I$). In this case, $w=0.5$. 

In [4]:
model,X_list, controls, true_vals, batch_effects,data_subtracted = run_experiment(2021)

Warmup:   0%|                                              | 0/21 [00:00, ?it/s]/home/austin/anaconda3/envs/bystro/lib/python3.10/site-packages/pyro/distributions/lkj.py:16: FutureWarning: class LKJCorrCholesky(d, eta, validate_args=None) is deprecated in favor of LKJCholesky(dim, concentration, validate_args=None).
  warnings.warn(
Sample: 100%|████| 21/21 [00:01, 20.74it/s, step size=1.35e-01, acc. prob=0.998]


Sample: 100%|████| 21/21 [00:00, 25.27it/s, step size=3.40e-01, acc. prob=0.904]


Sample: 100%|████| 21/21 [00:00, 24.45it/s, step size=1.47e-01, acc. prob=0.759]


Sample: 100%|████| 21/21 [00:00, 46.31it/s, step size=2.70e-01, acc. prob=0.001]


Sample: 100%|████| 21/21 [00:00, 29.34it/s, step size=2.99e-01, acc. prob=0.001]


...

Sample: 100%|████| 21/21 [00:00, 35.43it/s, step size=7.04e-01, acc. prob=0.000]


Sample: 100%|████| 21/21 [00:00, 35.70it/s, step size=4.76e-01, acc. prob=0.000]


Sample: 100%|████| 21/21 [00:00, 35.55it/s, step size=3.12e-01, acc. prob=0.000]


Sample: 100%|████| 21/21 [00:00, 31.87it/s, step size=3.05e-01, acc. prob=0.132]

Ours 1.687643322547037
Ignored 1.8987285660395787
Naive 2.1695650063784666
[0.19513855 0.28190353 0.34446911]


So the MSE of our estimates of $\delta$ was $1.69$ as opposed to the case where we ignore batch effects ($1.89$) and the case where we subtract off the control sample ($2.17$). This improvement is the optimal case for improving on both schemes, as no batch effects will be slightly worse for ours but dramatically better than the "subtract control sample approach" and massive batch effects means that we will slightly underperform subtracting the control sample since measurement noise effect is minimal.

## Batch effects are dominant

Here $\Sigma_\theta=16I$ while $\Sigma_\epsilon=I$. We expect that the traditional subtract the control should do pretty well.

In [5]:
model_b,X_b, controls_b, true_vals_b, batch_effects_b,data_subtracted_b = run_experiment(2021,
                                                                                        sig_theta=4.0)

Sample: 100%|████| 21/21 [00:00, 21.25it/s, step size=3.83e-01, acc. prob=0.000]


Sample: 100%|████| 21/21 [00:00, 24.97it/s, step size=4.47e-01, acc. prob=0.000]


Sample: 100%|████| 21/21 [00:00, 40.18it/s, step size=2.06e-01, acc. prob=0.973]


Sample: 100%|████| 21/21 [00:00, 26.52it/s, step size=1.92e-01, acc. prob=0.640]


Sample: 100%|████| 21/21 [00:00, 24.89it/s, step size=1.24e-01, acc. prob=0.985]


...

Sample: 100%|████| 21/21 [00:00, 24.73it/s, step size=2.66e-01, acc. prob=0.256]


Sample: 100%|████| 21/21 [00:00, 26.17it/s, step size=1.35e-01, acc. prob=0.067]


Sample: 100%|████| 21/21 [00:00, 25.75it/s, step size=2.69e-01, acc. prob=0.000]


Sample: 100%|████| 21/21 [00:00, 21.85it/s, step size=2.41e-01, acc. prob=0.190]

Ours 2.2384253168911856
Ignored 4.515188343497704
Naive 2.1695650063784666
[0.55056074 0.66488634 0.69262991]


As expected, we are basically the same performance with the "subtract the control sample" approach.

## Measurement noise is dominant

In [6]:
model_b,X_b, controls_b, true_vals_b, batch_effects_b,data_subtracted_b = run_experiment(2021,
                                                                                        sig_theta=.1)

Sample: 100%|████| 21/21 [00:00, 40.75it/s, step size=1.09e+00, acc. prob=0.000]


Sample: 100%|████| 21/21 [00:00, 24.77it/s, step size=3.06e-01, acc. prob=0.384]


Sample: 100%|████| 21/21 [00:00, 36.90it/s, step size=5.06e-01, acc. prob=0.000]


Sample: 100%|████| 21/21 [00:00, 35.73it/s, step size=4.34e-01, acc. prob=0.024]


Sample: 100%|████| 21/21 [00:00, 25.90it/s, step size=2.81e-01, acc. prob=0.859]


...

Sample: 100%|████| 21/21 [00:00, 32.66it/s, step size=6.39e-01, acc. prob=0.000]


Sample: 100%|████| 21/21 [00:00, 27.49it/s, step size=2.71e-01, acc. prob=0.012]


Sample: 100%|████| 21/21 [00:00, 25.16it/s, step size=3.74e-01, acc. prob=0.011]


Sample: 100%|████| 21/21 [00:00, 26.63it/s, step size=7.64e-01, acc. prob=0.858]


Ours 1.1570127181844128
Ignored 1.105450095313051
Naive 2.1695650063784666
[0.16916874 0.19687359 0.24726584]


## Non-iid data.

In our final experiment we evaluate what happens when our data are not iid. In this demonstration we add an effect +3 to the first batch. It doesn't make a difference from the point of view of our algorithm, but presumably if these are all the controls for some disease, this effect is the scientific effect we are trying to detect.


In [9]:
def run_experiment_unbalanced(seed,sig_theta=1.0,sig_eps=1.0):
    X_list, controls, true_vals, batch_effects = simulate_data(seed=seed,sig_theta=sig_theta,sig_eps=sig_eps)
    
    X_list[0] += 3
    true_vals[0] += 3

    model = BatchAdaptationBayesian(n_burn=300,n_samples=300)
    data_altered = model.fit_transform(X_list, controls)

    data_altered_stack = np.vstack(data_altered)
    data_original_stack = np.vstack(X_list)
    true_stack = np.vstack(true_vals)

    data_subtracted = [X_list[i] - controls[i] for i in range(len(X_list))]
    data_subtracted_stack = np.vstack(data_subtracted)

    error_adapted = np.mean((data_altered_stack - true_stack) ** 2)
    error_original = np.mean((data_original_stack - true_stack) ** 2)
    error_subtracted = np.mean((data_subtracted_stack - true_stack) ** 2)
    print('Ours',error_adapted)
    print('Ignored',error_original)
    print('Naive',error_subtracted)
    print(model.w)

    return model,X_list, controls, true_vals, batch_effects,data_subtracted


In [10]:
model,X_list, controls, true_vals, batch_effects,data_subtracted = run_experiment_unbalanced(2021)

Sample: 100%|████| 21/21 [00:00, 34.35it/s, step size=8.51e-02, acc. prob=0.482]


Sample: 100%|████| 21/21 [00:00, 26.67it/s, step size=2.49e-01, acc. prob=0.193]


Sample: 100%|████| 21/21 [00:00, 21.09it/s, step size=2.46e-01, acc. prob=0.007]


Sample: 100%|████| 21/21 [00:00, 21.48it/s, step size=1.85e-01, acc. prob=0.985]


Sample: 100%|████| 21/21 [00:01, 17.30it/s, step size=1.26e-01, acc. prob=0.872]


Sample: 100%|████| 21/21 [00:00, 21.29it/s, step size=1.89e+00, acc. prob=0.000]

Ours 1.6978476069760042
Ignored 1.8987285660395787
Naive 2.1695650063784666
[0.17100422 0.30235832 0.41339891]


Once again we do better. But more importantly, let's make sure the elements in the first batch have higher values than the remaining batches.

In [14]:
print('Average value, adapted:',np.mean(data_subtracted[0]))
print('Average value, truth set:',np.mean(true_vals[0]))
print('Average value, measured:',np.mean(X_list[0]))

Average value, adapted: 2.54036207850536
Average value, truth set: 2.942432191463585
Average value, measured: 2.3756306286889224


And to compare to another batch

In [15]:
print('Average value, adapted:',np.mean(data_subtracted[1]))
print('Average value, truth set:',np.mean(true_vals[1]))
print('Average value, measured:',np.mean(X_list[1]))

Average value, adapted: 0.3850903439365517
Average value, truth set: 0.03641952421240738
Average value, measured: 0.17833629580401883


## Conclusions

When the measurement noise is comparable we outperform either naive subtraction or ignoring batch effects. When the noise is either far greater or far less we only have slight performance degredation. But the advantage is that we can estimate the size of these effects. Our method is also robust to imbalances in case/control.